In [ ]:
'''
Pytorch中模型训练步骤还是非常清晰的：

数据载入及处理

模型定义

超参数设置（损失函数定义、优化器定义、训练轮数）

训练模型

读取一个batch的数据，并前向传播
计算损失值
反向传播计算梯度
优化器优化模型
循环执行上述过程直到规定轮数
评估模型（非必须）

测试模型
'''

In [ ]:
import torch
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


print(torch.backends.mps.is_available()) #m1 GPU available?
print(torch.backends.mps.is_built()) #m1 GPU available?

In [ ]:
#基本運算單位為Tensor,為一個多維度的矩陣,可和numpy互轉

#建立tensor
w1=torch.tensor([[1,2],[3,4],[5,6]])
w2=torch.tensor([[2,2],[2,2],[2,2]])
x=torch.empty(2,3) #創建空(未初始化)的tensor(2列3行)
print(x.dtype) #資料型態
print(x.size())
y=torch.ones(2)
z=torch.zeros(2,3,dtype=torch.int)
a=torch.randn(2,3,5)

# + - * / 皆為element-wise
ans1=w1+w2 
ans2=w1*w2 
print(ans1,ans2,sep='\n')

#矩陣乘法
x1=torch.tensor([[1,2],[3,4],[5,6]])
x2=torch.tensor([[1,2,3],[4,5,6]])
ans=x1.mm(x2) #二為矩陣乘法
print(ans)
ans=torch.mm(x1,x2) #兩個相等
print(ans)

#三維帶batch矩陣乘法：

#torch.bmm(a,b),tensor a 的size为(b,h,w),tensor b的size为(b,w,m) 
#也就是说两个tensor的第一维是相等的，然后第一个数组的第三维和第二个数组的第二维度要求一样，
#对于剩下的则不做要求，输出维度 （b,h,m）

#選取部分資料

print(w1[0,:]) #取得第一個row的tensor
print(w1[1:,:]) #取得第二個row以後的所有tensor
print(x[:,0]) #取得第ㄧcolumn的tensor
print(x[0,0]) #取得第一row,第一column的tensor
print(x[0,0].item()) #取得第一row,第一column的item
y1=y.numpy() #tensor to numpy
z1=z.view(3,2) #reshape
z2=z.reshape(3,2) #reshape和view功能基本一樣
z3=z.reshape(3,-1) #-1代表這個維度的個數是剩下的所有元素個數


In [ ]:
#讀取資料
#(1)圖片放在同一文件夾內,且另有一文件顯示標籤
#(2)不同類別的圖片放在不同的文件夾內,且文件夾就是圖片的類別
#(3)用torchvision來下載


#(1)
#需自定義一class來繼承torch.utils.data.dataset,並且要override 3個method
#Dataset配Dataloader

from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
import os
from PIL import Image
import numpy as np

class TorchDataset(Dataset):
    def __init__(self,filename,image_dir): 
        # --------------------------------------------
        # Initialize paths, transforms, and so on
        # --------------------------------------------
        
        #filename:数据文件TXT：格式：imge_name.jpg label_id
        #image_dir:图片路径：image_dir+imge_name.jpg构成图片的完整路径
        
        self.image_label_list=self.read_file(filename) #所有的name和label(type:tuple),用list存
        self.image_dir=image_dir #不完整的地址
        #self.len=len(self.image_label_list)
    
    def img_loader(self,image_path):
        image = Image.open(image_path)
        return image.convert('RGB')
        
    def __getitem__(self, index):
        # --------------------------------------------
        # 1. Read one data from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data pair (e.g. image and label)
        # --------------------------------------------
        image_name,label=self.image_label_list[index] #tuple的讀取
        print(self.image_label_list[index])
        image_path=os.path.join(self.image_dir,image_name) #完整地址
        print(image_path)
        #下面三行是前處理
        img = self.img_loader(image_path) 
        tranform = transforms.Compose([transforms.Resize((214, 214)), transforms.ToTensor()])
        img = tranform(img)
        label=np.array(label)
        return img,label
        
    def __len__(self):
        # --------------------------------------------
        # Indicate the total size of the dataset (資料的筆數)
        # --------------------------------------------
        return len(self.image_label_list)

    def read_file(self, filename): 
        image_label_list = []
        with open(filename, 'r') as f:
            lines = f.readlines()
            for line in lines:
                # rstrip：用来去除结尾字符、空白符(包括\n、\r、\t、' '，即：换行、回车、制表符、空格)
                content = line.rstrip().split(',') #是','而不是' ',可能因為是csv檔的關係
                name = content[0]
                labels = []
                for value in content[1:]:
                    labels.append(int(value))
                image_label_list.append((name, labels)) #用tuple的方式存到list裡
        return image_label_list
    
train_filename='/Users/sleepy/Documents/python/data/dogcat.csv'
image_dir='/Users/sleepy/Documents/python/data/cat&dog'
batch_size=9
epoch_num=2
train_data = TorchDataset(filename=train_filename, image_dir=image_dir) #資料打包起來
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True) #如何取樣資料

#DataLoader:用來產生批次訓練數據
#DataLoader(dataset,batch_size=1,shuffle=False,sampler=None,num_workers=0,collate_fn,pin_memory,drop_last=False)
for epoch in range(epoch_num):
    for batch_image, batch_label in train_loader:
        image=batch_image[0,:]
        image=image.numpy()#image=np.array(image)
        #image = image.transpose(1, 2, 0)  # 通道由[c,h,w]->[h,w,c]
        #image_processing.cv_show_image("image",image)
        print("batch_image.shape:{},batch_label:{}".format(batch_image.shape,batch_label))
        # batch_x, batch_y = Variable(batch_x), Variable(batch_y)

    
"""

#(2)
#調用torchvision.datasets.ImageFolder來處理

#ImageFolder(root,transform=None,target_transform=None,loader=default_loader)
#  transform:對PIL image 進行轉換操作
#  target_transform:對label進行轉換
#  loader:default為讀取PIL image
#
#  Attributes:
#  (1)classes: List of class names
#  (2)class_to_idx: Dict with items(class_name,class_index)
#  (3)imgs: List of (img path,class_index) tuples

#DataLoader:用來產生批次訓練數據
#DataLoader(dataset,batch_size=1,shuffle=False,sampler=None,num_workers=0,collate_fn,pin_memory,drop_last=False)

from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms

transform = transforms.Compose([transforms.Resize((214, 214)), transforms.ToTensor()])
trainset = ImageFolder('/Users/sleepy/Documents/python/data', transform=transform)
print(trainset)
print(trainset.class_to_idx)
#trainset[0][0]:第一維度代表第幾張圖片第二維度為0代表圖片數據
#trainset[0][1]:第二維度為1代表label(0,1,2,...)
print(trainset[300][1])

data_loader=torch.utils.data.DataLoader(trainset,batch_size=20,shuffle=True)

"""

#(3)
data_train = datasets.MNIST(root = "./data/",
                            transform=transform,
                            train = True,
                            download = True)

#root指定了数据集存放的路径，transform指定导入数据集时需要进行何种变换操作，train设置为True说明
#导入的是训练集合，否则为测试集合。train如果为true，则从internet下载数据集并将其放在根目录中。如
#果数据集已下载，则不会再次下载。

data_loader_train = torch.utils.data.DataLoader(dataset=data_train,
                                                batch_size = 64,
                                                shuffle = True)





In [ ]:
#建構model
#使用torch.nn.Module，我们可以根据自己的需求改变传播过程，如RNN等
#如果你需要快速构建或者不需要过多的过程，直接使用torch.nn.Sequential即可。

#法1
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )

#法2
#繼承了torch.nn.Module,並需要override __init__和forward

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()  #python3可以直接用super().xxx代替super(class_name,self).xxx
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = self.predict(x)
        return x

net1 = Net(1, 10, 1)

In [ ]:
#設定損失函數和優化器
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #用來算gredient decent 第一個參數指定了什么参数应当被优化

In [ ]:
#訓練

def train( train_loader, val_loader, epochs, model, criterion, optimizer, Print_info=True ):
    
    train_loss_list = []
    #val_loss_list = []
    
    model.train() #设置模型为训练模式,启用 batch normalization 和 dropout 。
    
    for epoch in range(epochs): #每一個epoch
        loss = 0    
        ###### Train ########
        for X,y in train_loader: #X,y:批次中的圖片和标签数据。 
            #也可for X,y in enumerate(train_loader, 0) 第二个参数表示迭代器的起始索引值，默认为0
            #enumerate()
            #>>> seq = ['one', 'two', 'three']
            #>>> for i, element in enumerate(seq):
            #...     print i, element
            #...
            #0 one
            #1 two
            #2 three
            #也可for batch_idx, (images, labels) in enumerate(train_loader,0): 
            #X = X.to(device)
            #y = y.to(device)
            
            #前向传播
            pred_y = model( X ) #当你调用一个类的实例对象时（即使用括号）(model是個object)，实际上是在调用该类的 __call__() 方法。
            
            计算损失函数
            loss = criterion(pred_y, y) #当你调用一个类的实例对象时（即使用括号），实际上是在调用该类的 __call__() 方法。
            #将梯度清零
            optimizer.zero_grad()
            #反向传播
            loss.backward()
            #更新模型参数
            optimizer.step()
            
            train_loss_list.append(loss.item()) #loss.item()可获取当前的损失值

        '''
        ###### Validation ######
        correct = 0
        with torch.no_grad():
            for X,y in val_dataloader:
                X = X.to(device)
                y = y.to(device)
                
                pred_y = model( X )
                loss = loss_fn(pred_y, y)
                val_loss_list.append(loss.item())
        '''
                       
    return train_loss_list